# Оценка рыночной стоимости мотоцикла с пробегом

## Выполнили: Болгарин Максим, Моисеенков Павел. BD-11

<br>

**Цель проекта:** проанализировать факторы, влияющие на рыночную стоимость подержанных мотоциклов, и научиться её предсказывать. Какую пользу можно вынести из этого простому человеку? Зная, как образуется стоимость мотоцикла, владелец подобного транспортного средства может подобрать подходящую для него цену для продажи. С другой стороны, будущий владелец сможет оценить адекватность заинтересовавшего его предложения.

**План выполнения:**
1. Поиск данных, формирование датасета
    * Скачивание и парсинг данных.
    * Формирование датасета.
2. Первичный анализ данных
    * Изучение данных: определение смысла признаков, типов данных, наличия пропусков и т.д.
3. Предобработка данных
    * Очистка и отбор данных, выполнение необходимых преобразований.
4. Визуальный анализ данных
    * Поиск скрытых особенностей в данных, анализ корреляций.
5. Применение моделей анализа данных для предсказания стоимости
    * Тестирование нескольких простых моделей, сравнение качества, выбор baseline.
    * Обучение и проверка модели, показывающей наилучшее качество.
6. Вывод

<br>

### 1. Поиск данных, формирование датасета

Источником данных послужил архив объявлений на сайте moto.drom.ru

Всего было обработано около 7000 объявлений из Москвы. Скачивание производилось в [другом Jupyter Notebook](https://github.com/maxbolgarin/datamining_project/blob/master/project/parce.ipynb). На выходе мы имеем .csv файл, с ним нам и предстоит работать.

#### Подключим необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

#### Теперь загрузим данные

In [2]:
motorcycles = pd.read_csv('data/motorcycles.csv', index_col='id')

In [3]:
motorcycles.sample(5)

,price,model,mileage,motorcycle_class,year,engine_capacity,engine_strokes,damaged,documents,city,date
id,,,,,,,,,,,
4151,185 000₽,Suzuki Djebel 250,37 000 км,None,2001,250 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,12 ноября 2015
4331,200 000₽,Suzuki GSF 1200 Bandit,42 500 км,None,2002,1 200 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,29 сентября 2015
1409,320 000₽,Hyosung GV650,7 000 км,Чоппер,2013,650 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,25 июня 2017
1166,129 000₽,Suzuki SV 400,30 000 км,Классика,2003,650 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,22 августа 2017
6827,27 000₽,Планета 5,11 000 км,None,1991,350 куб. см.,None,Исправен,Есть ПТС,Москва,20 мая 2010


In [4]:
motorcycles.shape

(6848, 11)

Видим, что данные успешно загрузились и выглядят вполне адекватно. Можем работать дальше.

<br>

### 2. Первичный анализ данных

Посмотрим общую информацию по датасету:

In [5]:
motorcycles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6848 entries, 1 to 7100
Data columns (total 11 columns):
price               6848 non-null object
model               6848 non-null object
mileage             6848 non-null object
motorcycle_class    6848 non-null object
year                6848 non-null object
engine_capacity     6848 non-null object
engine_strokes      6848 non-null object
damaged             6848 non-null object
documents           6848 non-null object
city                6848 non-null object
date                6848 non-null object
dtypes: object(11)
memory usage: 642.0+ KB


In [6]:
motorcycles.nunique()

price               1059
model               2284
mileage             1958
motorcycle_class      15
year                  56
engine_capacity      348
engine_strokes         3
damaged                3
documents              3
city                   1
date                2133
dtype: int64

In [7]:
motorcycles.drop_duplicates().shape

(6797, 11)

Теперь подробнее рассмотрим столбцы:

**price:**

In [8]:
motorcycles.price.value_counts()

150 000₽           135
250 000₽           132
200 000₽           132
300 000₽           115
350 000₽           109
220 000₽           103
120 000₽           101
180 000₽            92
100 000₽            89
160 000₽            86
190 000₽            86
110 000₽            85
Цена не указана     85
170 000₽            83
450 000₽            83
230 000₽            83
130 000₽            80
280 000₽            74
140 000₽            72
240 000₽            71
165 000₽            71
400 000₽            69
90 000₽             63
260 000₽            60
125 000₽            58
320 000₽            58
270 000₽            58
210 000₽            58
500 000₽            55
290 000₽            54
                  ... 
1 135 000₽           1
18 800₽              1
207 800₽             1
169 990₽             1
91 000₽              1
879 000₽             1
317 000₽             1
611 000₽             1
719 000₽             1
999 900₽             1
69 999₽              1
378 000₽             1
479 000₽   

**model:**

In [9]:
motorcycles.model.value_counts()[:8]

Yamaha YZF R6             95
Honda CBR 600RR           73
Yamaha YZF R1             58
Suzuki Boulevard M109R    58
Yamaha FZ 6               55
Yamaha                    52
Kawasaki Ninja ZX-6R      51
Honda CB 400SF            51
Name: model, dtype: int64

**mileage:**

In [10]:
motorcycles.mileage.value_counts()[:10] / motorcycles.mileage.size

None         0.088201
1 км         0.068633
30 000 км    0.017377
20 000 км    0.016063
25 000 км    0.014895
40 000 км    0.010952
12 000 км    0.010806
15 000 км    0.009930
23 000 км    0.009784
10 000 км    0.009784
Name: mileage, dtype: float64

**motorcycle_class:**

In [11]:
motorcycles.motorcycle_class.value_counts() / motorcycles.motorcycle_class.size

None               0.518400
Чоппер             0.178592
Классика           0.067465
Спортивный         0.053154
Круизер            0.040888
Спорт-турист       0.037967
Эндуро             0.025263
Питбайк            0.024971
Туристический      0.018692
Стритфайтер        0.011536
Кроссовый          0.008470
Кастом             0.007155
Мотард             0.003213
Детский            0.002482
Трайк (трицикл)    0.001752
Name: motorcycle_class, dtype: float64

**year:**

In [12]:
motorcycles.year.value_counts()[:10] / motorcycles.year.size

2014    0.070093
2013    0.063960
2008    0.062354
2006    0.059725
2007    0.058557
2005    0.047605
2012    0.045853
2009    0.044539
2003    0.043954
None    0.043808
Name: year, dtype: float64

**engine_capacity:**

In [13]:
motorcycles.engine_capacity.value_counts()[:13] / motorcycles.engine_capacity.size

400 куб. см.      0.110981
600 куб. см.      0.097985
250 куб. см.      0.088347
1 000 куб. см.    0.049504
650 куб. см.      0.045999
750 куб. см.      0.036069
1 200 куб. см.    0.035193
1 300 куб. см.    0.032856
125 куб. см.      0.027599
1 800 куб. см.    0.025847
1 100 куб. см.    0.025701
800 куб. см.      0.024533
None              0.019860
Name: engine_capacity, dtype: float64

**engine_strokes:**

In [14]:
motorcycles.engine_strokes.value_counts() / motorcycles.engine_strokes.size

4х тактный    0.863902
None          0.081484
2х тактный    0.054614
Name: engine_strokes, dtype: float64

**damaged:**

In [15]:
motorcycles.damaged.value_counts() / motorcycles.damaged.size

Исправен      0.968166
None          0.019568
Неисправен    0.012266
Name: damaged, dtype: float64

**documents:**

In [16]:
motorcycles.documents.value_counts() / motorcycles.documents.size

Есть ПТС    0.908732
Без ПТС     0.089223
None        0.002044
Name: documents, dtype: float64

**date:**

In [17]:
motorcycles.sample(10).date

id
3756       28 марта 2016
2970     26 августа 2016
2582     27 октября 2016
989       5 октября 2017
2592     23 октября 2016
4339    28 сентября 2015
2647     10 октября 2016
818            24 января
5079      19 января 2015
2195      12 апреля 2017
Name: date, dtype: object

Пройдемся по столбцам и, на основе уже имеющейся информации, дадим им некоторое описание:
1. **id:** просто порядковый номер мотоцикла, который вряд ли несет какую либо полезную информацию.
2. **price:** цена мотоцикла, целевая переменная. Видим, что есть объекты со значением "Цена не указана". Их нам придется отбросить. Также можно заметить, что не у всех мотоциклов стоит адекватная цена (290₽ и т.п.). Такие объекты тоже скорее всего можно отбросить, так как они вряд ли составляют весомую долю от всех остальных. Также следует преобразовать стоимость в числовой тип, отбросив символ "₽".
3. **model:** модель мотоцикла. Можно построить отдельный столбец **manufacturer**, в котором указывать только производителя, и рассматривать его как новое свойство.
4. **mileage:** пробег мотоцикла. 9% имею пропуски, возможно это из за того, что такие мотоциклы являются новыми. Было бы странно, если бы продавец не указывал хотя бы примерный пробег своего подержанного транспортного средства, ведь это является один из самых главных факторов при покупке. Так что заполним пропуски нулями. Также 7% имеют пробег "1 км", это странное значение можно тоже занулить. Значения стоит привести к числовым, отбросить подстроку "км".
5. **motorcycle_class:** класс мотоцикла. Как видим, около 50% всех объектов не имеют информции о классе. Возможно, этот столбец придется убрать из рассмотрения, так как заполнить пропуски сложно без искажения реальной информации. Считать отсутстиве значения как свойство не имеет смысла, так как класс присущ каждому мотоциклу, и отстуствие информации об этом характеризует скорее не мотоцикл, а человека, составляющего объявление.
6. **year:** год выпуска мотоцикла. Как видим, значения отсутсвуют лишь у 4% всех объявлений, что не значительно.
7. **engine_capacity:** объем двигателя. Пропуски имеют 2% объектов. Значения можно преобразовать с числовой тип, отбросив подстроку "куб. см.".
8. **engine_strokes:** число тактов двигателя. Видим, что всего 2 разных значения, и что больше 85% мотоциклов обладают четырехтактным двигателем. Также имеем 8% пропусков. Можно попробовать заменить пропуски на "4х тактный", так как большинство двигателей именно такие, и скорее всего продавец скорее всего знал бы, что у него особенный "2х тактный" двигатель, и указал бы это. Не думаю, что ошибка будет более 1% от общего количества. Также обрежем строку до одного числа, 2 или 4.
9. **damaged:** состояние мотоцикла. Пропуски имеют 2%, и с большой уверенностью можно заполнить их значениями "Исправен", так как предполагается, что если мотоцикл сломан, то продавец об этом скорее всего сообщит заранее (если он добросовестный, конечно). Также значение "Исправен" можно закодировать под 0, а "Неисправен" - под 1.
10. **documents:** наличие ПТС у продавца. Аналогично с прошлым столбцом, можно заполнить пропуски самым популярным вариантом и закодировать "Есть ПТС" как 0 и "Без ПТС" как 1.
11. **city:** все объявления из одного города, не несет информации.
12. **date:** как можно заметить из семпла, у некоторых объявлений есть информация о годе, а у некоторых нет. Те, у которых нет - объявления за 2018 год. Надо подправить это и дописать 2018 к этим датам.

<br>

### 3. Предобработка данных

Для начала избавимся от повторяющихся значений и запомним изначальное число объявлений:

In [18]:
motorcycles = motorcycles.drop_duplicates()
number_of_motocycles = motorcycles.shape[0]

Напишем функцию drop_none для упрощения работы:

In [19]:
def drop_none(df):
    return df.replace(to_replace='None', value=np.nan).dropna()

Теперь произведем обработку слобцов в соответствии с замечаниями, данными выше.

Начнем с преобразования целевого столбца **price**:

In [20]:
motorcycles.price = drop_none(motorcycles.price)
motorcycles = motorcycles[motorcycles.price != "Цена не указана"]
motorcycles.price = motorcycles.price.str[:-1].replace(' ', '', regex=True).astype(np.int64)
motorcycles.price = motorcycles.price[motorcycles.price.values > 4999]

Перейдем к столбцу **model**:

In [21]:
(motorcycles.model.shape[0] - drop_none(motorcycles.model).shape[0]) / motorcycles.model.shape[0]

0.0010429082240762813

Как видим, None имеют один процент мотоциклов, поэтому отбросим их и создадим новый столбец **manufacturer**:

In [22]:
motorcycles.model = drop_none(motorcycles.model)
motorcycles['manufacturer'] = [item[0] if type(item) is list else item
                               for item in motorcycles.model.str.split(' ').values]

Преобразуем столбец **mileage**:

In [23]:
motorcycles.mileage = motorcycles.mileage.str.replace("None", "0")
motorcycles.mileage = motorcycles.mileage.str.replace("1 км", "0", regex=True)
motorcycles.mileage = motorcycles.mileage.str.replace("км", "", regex=True)
motorcycles.mileage = motorcycles.mileage.str.replace(' ', '', regex=True).astype(np.int64)

Как решили ранее, с **motorcycle_class** работать сложно, поэтому просто не будем рассматривать этот столбец.

In [24]:
motorcycles = motorcycles.drop(['motorcycle_class'], axis=1)

Отбросим пропуски и преобразуем тип значений столбца **year** в integer:

In [25]:
motorcycles.year = drop_none(motorcycles.year)
motorcycles.year = motorcycles.year.dropna().astype(np.int32)

На очереди **engine_capacity**:

In [26]:
motorcycles.engine_capacity = drop_none(motorcycles.engine_capacity)
motorcycles.engine_capacity = motorcycles.engine_capacity.str.replace("куб. см.", '', regex=True)
motorcycles.engine_capacity = motorcycles.engine_capacity.str.replace(' ', '', regex=True)
motorcycles.engine_capacity = motorcycles.engine_capacity.dropna().astype(np.int64)

В столбце **engine_strokes** заменим None на "4х тактный":

In [27]:
motorcycles.engine_strokes = motorcycles.engine_strokes.str.replace("None", "4", regex=True)
motorcycles.engine_strokes = motorcycles.engine_strokes.str.replace("х тактный", '', regex=True)
motorcycles.engine_strokes = motorcycles.engine_strokes.dropna().astype(np.int64)

Преобразуем **damaged**:

In [28]:
motorcycles.damaged = motorcycles.damaged.str.replace("None", "0", regex=True)
motorcycles.damaged = motorcycles.damaged.str.replace("Исправен", "0", regex=True)
motorcycles.damaged = motorcycles.damaged.str.replace("Неисправен", "1", regex=True)
motorcycles.damaged = motorcycles.damaged.dropna().astype(np.int64)

Аналогично **documents**:

In [29]:
motorcycles.documents = motorcycles.documents.str.replace("None", "0", regex=True)
motorcycles.documents = motorcycles.documents.str.replace("Есть ПТС", "0", regex=True)
motorcycles.documents = motorcycles.documents.str.replace("Без ПТС", "1", regex=True)
motorcycles.documents = motorcycles.documents.dropna().astype(np.int64)

Выкидываем за ненадобностью столбец **city**:

In [30]:
motorcycles = motorcycles.drop(['city'], axis=1)

Наконец, преобразуем столбец **date**. Сначала добавим год тем объектам, у которых его нет:

In [31]:
motorcycles.date = [s + " 2018" if len(s.split()) == 2 else s for s in motorcycles.date.values]

По хорошему, строки с датой надо бы привести к формату datetime. Однако, не совсем понятно, как работать с названиями месяцев на русском языке. Поэтому поменяем их на английский, а потом приведем значения к datetime:

In [32]:
rus_eng_month = {
    'января': 'January',
    'февраля': 'February',
    'марта': 'March',
    'апреля': 'April',
    'мая': 'May',
    'июня': 'June',
    'июля': 'July',
    'августа': 'August',
    'сентября': 'September',
    'октября': 'October',
    'ноября': 'November',
    'декабря': 'December',  
}

dates = []
for s in motorcycles.date.values:
    temp_s = s.split()
    dates.append(temp_s[0] + ' ' + rus_eng_month[temp_s[1]] + ' ' + temp_s[2])
    
motorcycles.date = [datetime.strptime(s, '%d %B %Y').strftime('%Y-%m-%d') for s in dates]
motorcycles.date = pd.to_datetime(motorcycles.date.dropna()).dt.date

Отбросим оставшиеся неопределенные значения:

In [33]:
motorcycles = drop_none(motorcycles)
motorcycles = motorcycles.dropna()

Вот так теперь выглядит таблица:

In [34]:
motorcycles.sample(5)

,price,model,mileage,year,engine_capacity,engine_strokes,damaged,documents,date,manufacturer
id,,,,,,,,,,
4135,90000.0,Honda CBR,15000,1997.0,900.0,4,0,1,2015-11-19,Honda
1292,690000.0,Honda VFR 1200F,20100,2013.0,1200.0,4,0,0,2017-07-19,Honda
7087,20000.0,Dio,0,2001.0,50.0,4,0,0,2009-07-26,Dio
394,430000.0,Honda DN-01,9000,2012.0,680.0,4,0,0,2018-06-20,Honda
1043,350000.0,Honda CBR 600RR,0,2007.0,600.0,4,0,0,2017-09-27,Honda


In [35]:
motorcycles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6334 entries, 1 to 7100
Data columns (total 10 columns):
price              6334 non-null float64
model              6334 non-null object
mileage            6334 non-null int64
year               6334 non-null float64
engine_capacity    6334 non-null float64
engine_strokes     6334 non-null int64
damaged            6334 non-null int64
documents          6334 non-null int64
date               6334 non-null object
manufacturer       6334 non-null object
dtypes: float64(3), int64(4), object(3)
memory usage: 544.3+ KB


Посмотрим, сколько осталось объектов после всех преобразований:

In [36]:
print('Было: {}'.format(number_of_motocycles))
print('Осталось: {}'.format(motorcycles.shape[0]))
print('Датасет уменьшился на {:.3}% от первоначального размера'.format(
    100*(number_of_motocycles - motorcycles.shape[0]) / number_of_motocycles)
     )

Было: 6797
Осталось: 6334
Датасет уменьшился на 6.81% от первоначального размера


Как видим, после всех преобразований, датасет уменьшился незначительно. Теперь мы можем приступать к основной счасти работы.

<br>